Reference : https://github.com/kokkoks/thai-joke-sentence-generator


Reference : https://kok-suttanan.medium.com/%E0%B8%AA%E0%B8%A3%E0%B9%89%E0%B8%B2%E0%B8%87-%E0%B8%A1%E0%B8%B8%E0%B8%81%E0%B9%80%E0%B8%AA%E0%B8%B5%E0%B9%88%E0%B8%A2%E0%B8%A7-%E0%B9%82%E0%B8%94%E0%B8%A2%E0%B9%83%E0%B8%8A%E0%B9%89-python-%E0%B9%81%E0%B8%A5%E0%B8%B0-deep-learning-text-generation-809a9c8686ab

In [3]:
!pip install pythainlp

# Step0 Load Library

In [4]:
import pandas as pd
import numpy as np
import string, os 
from pythainlp import word_tokenize

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

# Step1 Explore data and preprocessing

In [5]:
trainDF = pd.read_csv("caption.txt",header=None,names=["text"])
trainDF.head()

,text
0,กฎข้อที่1 จงอย่าขาดทุน
1,การตั้งเป้าหมายที่ดี เท่ากับสำเร็จแล้วครึ่งนึง
2,“ความล้มเหลว” ก็คือ “ความสำเร็จ” ถ้าหากเรา “เร...
3,คุณสามารถทำมันให้ดีขึ้นได้อีกในทุกๆวัน
4,จงรักในหุ้น แต่อย่าภักดีในราคา


In [6]:
trainDF["tokens"] = trainDF["text"].apply(lambda text: word_tokenize(text, keep_whitespace=True))

In [7]:
trainDF["processed_tokens"] = trainDF["tokens"].apply(lambda tokens: tokens + ["END"])
trainDF["processed_sentence"] = trainDF["processed_tokens"].apply(lambda tokens: " ".join(tokens))
trainDF[["processed_sentence"]]

,processed_sentence
0,กฎ ข้อ ที่ 1 จง อย่า ขาดทุน END
1,การ ตั้งเป้าหมาย ที่ ดี เท่ากับ สำเร็จ แล้ว...
2,“ ความล้มเหลว ” ก็ คือ “ ความสำเร็จ ” ถ้...
3,คุณ สามารถ ทำ มัน ให้ ดีขึ้น ได้ อีก ใน ทุกๆ ว...
4,จงรัก ใน หุ้น แต่ อย่า ภักดี ใน ราคา END
...,...
193,มืออาชีพ ไม่ต้อง การงาน ที่ หนัก ความ ฉลาด ควา...
194,เมื่อ คุณ รู้ ว่า จะ ไม่ ทำ อะไร เพื่อ เสียเงิ...
195,การซื้อขาย ที่ ประสบความสำเร็จ นั้น เป็น การต่...
196,ประโยค ที่ อันตราย ที่สุด ใน การลงทุน ก็ คือ ...


In [8]:
tokenizer = Tokenizer(lower=False)

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(trainDF["processed_sentence"])
tokenizer.sequences_to_texts(inp_sequences[:10])

['กฎ ข้อ',
 'กฎ ข้อ ที่',
 'กฎ ข้อ ที่ 1',
 'กฎ ข้อ ที่ 1 จง',
 'กฎ ข้อ ที่ 1 จง อย่า',
 'กฎ ข้อ ที่ 1 จง อย่า ขาดทุน',
 'กฎ ข้อ ที่ 1 จง อย่า ขาดทุน END',
 'การ ตั้งเป้าหมาย',
 'การ ตั้งเป้าหมาย ที่',
 'การ ตั้งเป้าหมาย ที่ ดี']

In [9]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors = input_sequences[:,:-1]
    label = input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
print(f"max_sequence_len: {max_sequence_len}, total_words: {total_words}")

max_sequence_len: 40, total_words: 730


# Step2 LSTM

In [10]:
from keras.layers import LSTM, Input, Bidirectional
# from keras.optimizers import Adam

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, output_dim=128, input_length=input_len))
    
    # model.add(LSTM(100))
    # model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dropout(0.5))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 39, 128)           93440     
                                                                 
 bidirectional (Bidirectiona  (None, 39, 512)          788480    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 39, 512)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 39, 512)          1574912   
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 39, 512)           0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              6

In [11]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.fit(predictors, label, epochs=500, verbose=1, callbacks=[early_stopping])

Epoch 1/500
86/86 [==============================] - 10s 28ms/step - loss: 5.9401
Epoch 2/500
86/86 [==============================] - 2s 26ms/step - loss: 5.6462
Epoch 3/500
86/86 [==============================] - 2s 28ms/step - loss: 5.6031
Epoch 4/500
86/86 [==============================] - 2s 26ms/step - loss: 5.5260
Epoch 5/500
86/86 [==============================] - 2s 26ms/step - loss: 5.4046
Epoch 6/500
86/86 [==============================] - 2s 26ms/step - loss: 5.3121
Epoch 7/500
86/86 [==============================] - 2s 27ms/step - loss: 5.2508
Epoch 8/500
86/86 [==============================] - 2s 28ms/step - loss: 5.1667
Epoch 9/500
86/86 [==============================] - 2s 28ms/step - loss: 5.0863
Epoch 10/500
86/86 [==============================] - 2s 28ms/step - loss: 5.0167
Epoch 11/500
86/86 [==============================] - 2s 28ms/step - loss: 4.9107
Epoch 12/500
86/86 [==============================] - 2s 28ms/step - loss: 4.8337
Epoch 13/500
86/86 [====

In [16]:
def generate_text(text, model, max_sequence_len):
    seed_text = " ".join(word_tokenize(text))
    counter = 0
    while counter < 100:
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted = np.random.choice(np.arange(0, predicted.shape[1]), p=predicted[0])

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
        if output_word == "END":
          joined_text = "".join(seed_text.split(" ")[:-1])
          return joined_text
        counter += 1
    return seed_text

In [23]:
test_text = "ลงทุน"
print(generate_text(test_text, model, max_sequence_len))

ลงทุนไป100แต่ผลตอบแทน0


In [18]:
import pickle

model.save("model.h5")
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)